# Run Delta segmentation & tracking pipeline

Here we will use the Delta2 package to segment and track timelapse data of microcolonies using a deep learning based workflow.  
You can find extensive documentation on Delta [here](https://delta.readthedocs.io).

In addition we will need the `json` package to edit the configuration files.

In [1]:
import pathlib
import delta
import json

Deep Learning networks can run on a CPU, but are much much faster on a GPU.  
Let's see if we can use a GPU (you should see get a line with `device_type='GPU'):

In [2]:
import tensorflow as tf
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

---

## Setup Folders
Set the path to the one you used in `0_download_model_delta`.  
We will also create a `ProcessedData` folder where we will store the output

In [3]:
root = pathlib.Path(pathlib.Path.home(), 'Documents/Image_analysis/Pseudomonas/2023.09.27_Psy_AgarPads#13')
strain_dir = (root / 'Psy-eGFP') 
model_dir = strain_dir / 'models' #location of model

!ls $strain_dir

0_Run_delta_pipeline_Psy.ipynb   2023.09.27_Psy_AgarPads#13_Pos63
2023.09.27_Psy_AgarPads#13_Pos41 2023.09.27_Psy_AgarPads#13_Pos69
2023.09.27_Psy_AgarPads#13_Pos42 2023.09.27_Psy_AgarPads#13_Pos71
2023.09.27_Psy_AgarPads#13_Pos44 2023.09.27_Psy_AgarPads#13_Pos75
2023.09.27_Psy_AgarPads#13_Pos47 RawData_from_microscope
2023.09.27_Psy_AgarPads#13_Pos49 config_2D.json
2023.09.27_Psy_AgarPads#13_Pos50 config_2D_local.json
2023.09.27_Psy_AgarPads#13_Pos51 config_mothermachine.json
2023.09.27_Psy_AgarPads#13_Pos55 extra_batch_script.ipynb
2023.09.27_Psy_AgarPads#13_Pos60 models
2023.09.27_Psy_AgarPads#13_Pos62


---

## Modify Config Files

The Delta Pipeline is controlled using `.json` [config files](https://delta.readthedocs.io/en/latest/usage/config_desc.html). By default two config files are provide: one for 1D mother machine data: `config_mothermachine.json` and one for 2D (microcolony or flowcell) data: `config_2D.json`.

You need to modify these config files to point Delta to the correct data folder.  
We also need to specify the correct path to the pre-trained models.

Here we will open the config file using the [`json`](https://docs.python.org/3/library/json.html) package:

In [5]:
# select config file to modify ('2D' or 'mothermachine'):
config_filename = strain_dir / 'config_2D.json'
with open(config_filename) as f:
    config = json.load(f)

Now we have to modify the relevant fields to point to the correct paths.

*Technical note: reminder: `delta` needs paths specified as strings, we can make a quick function to do this:*

In [6]:
def to_str(posixpath):
    return str(posixpath.resolve())    

In [7]:
#point to model location
config['model_file_seg'] = to_str(model_dir / 'unet_pads_seg.hdf5')    
config['model_file_track'] = to_str(model_dir / 'unet_pads_track.hdf5')    

#only needed for model training, in that case, point to location of training data
config['training_set_seg'] = ''
config['training_set_track'] = ''

#point to raw data location
config['eval_movie'] = '' #to_str(data_dir)

#specify output formats
config['save_format'] = ["pickle", "movie"]    

Now we can write the config file back to disk:

In [8]:
new_config_filename = strain_dir / 'config_2D_local.json'
with open(new_config_filename, 'w') as f:
    json.dump(config, f, indent=2)

---

## Batch process multiple positions for each strain

In [8]:
folder_names = sorted(strain_dir.glob('*Pos*'))
for folder in folder_names: print(folder.name)   

2023.09.27_Psy_AgarPads#13_Pos41
2023.09.27_Psy_AgarPads#13_Pos42
2023.09.27_Psy_AgarPads#13_Pos44
2023.09.27_Psy_AgarPads#13_Pos47
2023.09.27_Psy_AgarPads#13_Pos49
2023.09.27_Psy_AgarPads#13_Pos50
2023.09.27_Psy_AgarPads#13_Pos51
2023.09.27_Psy_AgarPads#13_Pos55
2023.09.27_Psy_AgarPads#13_Pos60
2023.09.27_Psy_AgarPads#13_Pos62
2023.09.27_Psy_AgarPads#13_Pos63
2023.09.27_Psy_AgarPads#13_Pos69
2023.09.27_Psy_AgarPads#13_Pos71
2023.09.27_Psy_AgarPads#13_Pos75


In [9]:
for folder in folder_names:

    posname = folder.name
    print('running position ', posname)
    
    #path to current position
    data_dir = strain_dir / posname #/ 'RawData'
    
    #make subfolder for current position
    output_dir = strain_dir / posname / 'ProcessedData'
    (output_dir).mkdir(exist_ok=True)  
    
    myprototype = 'C%01i-'+posname+'_crop%04i.tif'
    
    try:            
        # Load config ('2D' or 'mothermachine'):
        delta.config.load_config(new_config_filename)

        # Init reader (use bioformats=True if working with nd2, czi, ome-tiff etc):
        xpreader = delta.utils.xpreader(
                    data_dir,
                    prototype = myprototype,
                    fileorder = 'ct',
                    filenamesindexing=1
                    )

        # Print experiment parameters to make sure it initialized properly:
        print("""Initialized experiment reader:
            - %d positions
            - %d imaging channels
            - %d timepoints"""%(xpreader.positions, xpreader.channels, xpreader.timepoints)
        )

        # Init pipeline:
        xp = delta.pipeline.Pipeline(xpreader, resfolder=output_dir)   

        # Run it (you can specify which positions, which frames to run etc):
        xp.process()
        
    except:
        print('skipping postion', posname)
    

running position  2023.09.27_Psy_AgarPads#13_Pos41
Loading configuration from: /Users/jluneau/Documents/Image_analysis/Pseudomonas/2023.09.27_Psy_AgarPads#13/Psy-eGFP/config_2D_local.json
Initialized experiment reader:
            - 1 positions
            - 2 imaging channels
            - 53 timepoints
Metal device set to: Apple M1 Pro
Mon Oct 16 23:53:10 2023, Position 0 - Starting pre-processing
Mon Oct 16 23:53:10 2023, Position 0 - Starting segmentation (53 frames)
62/62 [==============================] - 10s 157ms/step
Mon Oct 16 23:54:06 2023, Position 0 - Starting tracking (53 frames)
Mon Oct 16 23:54:06 2023, Position 0 - Tracking - frame 0/53 
Mon Oct 16 23:54:06 2023, Position 0 - Tracking - frame 1/53 
Mon Oct 16 23:54:09 2023, Position 0 - Tracking - frame 2/53 
Mon Oct 16 23:54:10 2023, Position 0 - Tracking - frame 3/53 
Mon Oct 16 23:54:10 2023, Position 0 - Tracking - frame 4/53 
Mon Oct 16 23:54:11 2023, Position 0 - Tracking - frame 5/53 
Mon Oct 16 23:54:11 2023, P

[ WARN:0@3316.140] global /Users/runner/miniforge3/conda-bld/libopencv_1658893788286/work/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/Users/jluneau/Documents/Image_analysis/Pseudomonas/2023.09.27_Psy_AgarPads#13/Psy-eGFP/2023.09.27_Psy_AgarPads#13_Pos69/C1-2023.09.27_Psy_AgarPads#13_Pos69_crop0001.tif'): can't open/read file: check file path/integrity
[ WARN:0@3316.142] global /Users/runner/miniforge3/conda-bld/libopencv_1658893788286/work/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/Users/jluneau/Documents/Image_analysis/Pseudomonas/2023.09.27_Psy_AgarPads#13/Psy-eGFP/2023.09.27_Psy_AgarPads#13_Pos71/C1-2023.09.27_Psy_AgarPads#13_Pos71_crop0001.tif'): can't open/read file: check file path/integrity


Tue Oct 17 00:48:06 2023, Position 0 - Starting pre-processing
Tue Oct 17 00:48:06 2023, Position 0 - Starting segmentation (50 frames)
44/44 [==============================] - 7s 150ms/step
Tue Oct 17 00:48:55 2023, Position 0 - Starting tracking (50 frames)
Tue Oct 17 00:48:55 2023, Position 0 - Tracking - frame 0/50 
Tue Oct 17 00:48:55 2023, Position 0 - Tracking - frame 1/50 
Tue Oct 17 00:48:58 2023, Position 0 - Tracking - frame 2/50 
Tue Oct 17 00:48:59 2023, Position 0 - Tracking - frame 3/50 
Tue Oct 17 00:48:59 2023, Position 0 - Tracking - frame 4/50 
Tue Oct 17 00:48:59 2023, Position 0 - Tracking - frame 5/50 
Tue Oct 17 00:48:59 2023, Position 0 - Tracking - frame 6/50 
Tue Oct 17 00:49:00 2023, Position 0 - Tracking - frame 7/50 
Tue Oct 17 00:49:00 2023, Position 0 - Tracking - frame 8/50 
Tue Oct 17 00:49:00 2023, Position 0 - Tracking - frame 9/50 
Tue Oct 17 00:49:01 2023, Position 0 - Tracking - frame 10/50 
Tue Oct 17 00:49:01 2023, Position 0 - Tracking - frame 1

---

## Setup the pipeline for 1 position

Now we setup the pipeline, most importantly, you have to specify the naming format using the `xpreader` function.  
See [here](https://delta.readthedocs.io/en/latest/usage/pipeline_desc.html) for detailed instructions. 


In [12]:
# Set paths
pos_dir = strain_dir / '2023.09.27_Psy_AgarPads#13_Pos71'
data_dir = pos_dir #location of raw data
output_dir = pos_dir / 'ProcessedData' #location of output data
(output_dir).mkdir(exist_ok=True) #create output data folder

# Load config
delta.config.load_config(new_config_filename)

# set path to raw data to analyze:
file_path = to_str(data_dir)

# Init reader and specify file naming scheme
posname = pos_dir.name

xpreader = delta.utils.xpreader(
            file_path,
            prototype = 'C%01i-'+posname+'_crop%04i.tif', #'C%01i-2023.06.14_Psy_AgarPads#5_Pos%01i_crop%04i.tif',
            fileorder = 'ct',
            filenamesindexing=1
            )

# Print experiment parameters to make sure it initialized properly:
print("""Initialized experiment reader:
    - %d positions
    - %d imaging channels
    - %d timepoints"""%(xpreader.positions, xpreader.channels, xpreader.timepoints)
)

# Init pipeline:
xp = delta.pipeline.Pipeline(xpreader, resfolder=output_dir)   

Loading configuration from: /Users/jluneau/Documents/Image_analysis/Pseudomonas/2023.09.27_Psy_AgarPads#13/Psy-eGFP/config_2D_local.json
Initialized experiment reader:
    - 1 positions
    - 2 imaging channels
    - 45 timepoints


In [13]:
# Run Pipeline
xp.process()

Tue Oct 17 20:29:46 2023, Position 0 - Starting pre-processing
Tue Oct 17 20:29:46 2023, Position 0 - Starting segmentation (45 frames)
52/52 [==============================] - 8s 144ms/step
Tue Oct 17 20:30:17 2023, Position 0 - Starting tracking (45 frames)
Tue Oct 17 20:30:17 2023, Position 0 - Tracking - frame 0/45 
Tue Oct 17 20:30:17 2023, Position 0 - Tracking - frame 1/45 
Tue Oct 17 20:30:19 2023, Position 0 - Tracking - frame 2/45 
Tue Oct 17 20:30:20 2023, Position 0 - Tracking - frame 3/45 
Tue Oct 17 20:30:20 2023, Position 0 - Tracking - frame 4/45 
Tue Oct 17 20:30:20 2023, Position 0 - Tracking - frame 5/45 
Tue Oct 17 20:30:20 2023, Position 0 - Tracking - frame 6/45 
Tue Oct 17 20:30:21 2023, Position 0 - Tracking - frame 7/45 
Tue Oct 17 20:30:21 2023, Position 0 - Tracking - frame 8/45 
Tue Oct 17 20:30:21 2023, Position 0 - Tracking - frame 9/45 
Tue Oct 17 20:30:21 2023, Position 0 - Tracking - frame 10/45 
Tue Oct 17 20:30:22 2023, Position 0 - Tracking - frame 1